In [14]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import math

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print('using device', device)

using device cpu


In [15]:
torch.cuda.empty_cache()

In [16]:
dataset = ControlsDataset()
dataset.labels.set_type("angular")
dataset.images.set_grayscale(True)
print("Data size", len(dataset.labels))
dataset.split()

Data size 11019
Total training stacks 35
Total validation stacks 9


In [17]:
net = ConvNet(color_channels = 1, outputs = 1, dataset = dataset).to(device)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

units after conv 512
conv parameters:  164576
fc parameters:  164353
number of parameters:  328929


In [18]:
epochs = 1
optimizer = optim.Adam(net.parameters(), lr=0.0001)
#criterion = nn.CrossEntropyLoss() # Changed from Mean-Squared to Cross-Entropy
criterion = nn.MSELoss()
net.report_period = 1
net.fit(device,epochs,optimizer,criterion)

0,1	Loss:0.04384	Allocated:0.0GB	Cached:0.0GB

0,2	Loss:0.06696	Allocated:0.0GB	Cached:0.0GB

0,3	Loss:0.05625	Allocated:0.0GB	Cached:0.0GB

0,4	Loss:0.05477	Allocated:0.0GB	Cached:0.0GB

0,5	Loss:0.06863	Allocated:0.0GB	Cached:0.0GB

0,6	Loss:0.07493	Allocated:0.0GB	Cached:0.0GB

0,7	Loss:0.04331	Allocated:0.0GB	Cached:0.0GB

0,8	Loss:0.06049	Allocated:0.0GB	Cached:0.0GB

0,9	Loss:0.06375	Allocated:0.0GB	Cached:0.0GB

0,10	Loss:0.03611	Allocated:0.0GB	Cached:0.0GB

0,11	Loss:0.053	Allocated:0.0GB	Cached:0.0GB

0,12	Loss:0.0657	Allocated:0.0GB	Cached:0.0GB

0,13	Loss:0.05934	Allocated:0.0GB	Cached:0.0GB

0,14	Loss:0.05839	Allocated:0.0GB	Cached:0.0GB

0,15	Loss:0.05376	Allocated:0.0GB	Cached:0.0GB

0,16	Loss:0.0623	Allocated:0.0GB	Cached:0.0GB

0,17	Loss:0.07217	Allocated:0.0GB	Cached:0.0GB

0,18	Loss:0.05269	Allocated:0.0GB	Cached:0.0GB

0,19	Loss:0.0574	Allocated:0.0GB	Cached:0.0GB

0,20	Loss:0.04646	Allocated:0.0GB	Cached:0.0GB

0,21	Loss:0.0709	Allocated:0.0GB	Cached:0.0GB

0,22	Lo

In [ ]:
# End of training Validation Test
net.load("snapshots/0.727_model.pt")
total, correct = net.score(device,dataset, "categorical")
print("{:.1f}% classified correctly".format(100*correct/total))

In [ ]:
# Beginning of training Validation Test
net.load("snapshots/3.002_model.pt")
total, correct = net.score(device,dataset, "categorical")
print("{:.1f}% classified correctly".format(100*correct/total))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Additional Info when using cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))